In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import timedelta
from datetime import datetime
import dask.array

import sys
sys.path.append('../')
import snowFun

In [2]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [3]:
### choose if you want to do only the 45 validation glaciers
validation_only = 1

# load rgi names that have been saved to the classified folder
rgis_folder = list(set( [ i[3:17] for i in os.listdir(folder_class) if i!='merged.vrt' ] ))

# open list of validation glaciers
all_validation_df = pd.read_csv(os.path.join(folder_AGVA, 'Validation', 'Validation Glaciers.csv'))

# get rgi names for given o2 region
rgis_o2 = rgi_gdf[rgi_gdf['O2Region']=='4']['RGIId'].values

# select which rgis to analyze
if validation_only:
    rgis_to_analyze = list( set(rgis_folder).intersection(set(all_validation_df['RGIId'].values)) )
else:
    # rgis_to_analyze = ["RGI60-01.09162"] # just a single rgi
    rgis_to_analyze = rgis_folder # everything that is available
#     rgis_to_analyze = list( set(rgis_folder).intersection(set(rgis_o2)) ) # all the rgis in the folder than are in this o2region

# get list of glacier area for each rgi
areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# make df
rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})

# sort however you want
rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# grab rgi names
rgis_to_analyze = rgis_to_analyze_df['RGIId'].values


print(len(rgis_to_analyze_df))
# print(rgis_to_analyze[:10])
# print(rgis_to_analyze_df[:10])

45


In [4]:
c = 0
for i in range(len(rgis_to_analyze)):
#     if c>0: continue
    
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]

    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area'].values[0]

#     if ga<500: continue
        
    # choose how much to coarsen (more coarse for bigger glaciers) 
    if ga>1000:
        scale=5
    elif ga>500:
        scale=3
    else:
        scale=1
    
    # set folder
    if validation_only:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation')
    else:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')
       
    # print progress
    print(f"\nStarting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga} km2")
    
    # grab just this rgi geometry and info
    rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
    single_geometry = rgi_single.geometry

    # open glacier mask
    glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto').glacier
    
    # coarsen if this is a big glacier
#     if ga>500:
#         glacier_mask = glacier_mask.coarsen({"x":scale, "y":scale}, boundary="trim").median(skipna=True).astype('uint8')
    
    # count pixels
    glacier_pixels = glacier_mask.sum().values

    # open obs_df
    obs_path = os.path.join(folder_save, 'Daily AAs', 'observed', f"S2_{rgiid}_observed.csv")
    obs_df = pd.read_csv(obs_path)
    
    # for each year, open the daily shadow data, count usable, save to df
    for y in [2018,2019,2020,2021,2022]:
        if ga>100: print(y)
        
        # define path to shadow data
        path_shadow = os.path.join(folder_save, 'Shadows', f"S2_{rgiid}_{y}_daily_shadows.nc")
        
        # open, changing the chunk size depending on glacier area
        if ga>1000:
            chunks={'time':1}
        elif ga>100:
            chunks={'time':10}
        else:
            chunks='auto'
            
        shadows = xr.open_dataset(path_shadow, chunks=chunks).to_array()
        
        # get dates
        dates = shadows.time.values
        dates = [str(d)[:10] for d in dates]
        
        # count usable fraction each date
        percent_usable_by_time = shadows.sum(dim=['x','y']).values[0] #.compute().values[0][0]
        
        # scale correctly
        percent_usable_by_time = percent_usable_by_time/glacier_pixels #*scale*scale
        
        # put these in the df       
        obs_df.loc[obs_df['Date'].isin(dates), 'terrain_shadows'] = (1-percent_usable_by_time).round(4)

    # format df and save
    obs_df.to_csv(obs_path, index=False)
    c+=1
    
print("Done!")


Starting 1 of 45: RGI60-01.10910  2.084 km2

Starting 2 of 45: RGI60-01.00787  2.126 km2

Starting 3 of 45: RGI60-01.23606  2.344 km2

Starting 4 of 45: RGI60-01.15253  2.551 km2

Starting 5 of 45: RGI60-01.03379  2.578 km2

Starting 6 of 45: RGI60-01.16719  2.681 km2

Starting 7 of 45: RGI60-01.17321  2.88 km2

Starting 8 of 45: RGI60-01.13462  3.206 km2

Starting 9 of 45: RGI60-01.13483  3.216 km2

Starting 10 of 45: RGI60-01.02584  3.441 km2

Starting 11 of 45: RGI60-01.03215  3.998 km2

Starting 12 of 45: RGI60-01.01666  4.243 km2

Starting 13 of 45: RGI60-01.12548  4.314 km2

Starting 14 of 45: RGI60-01.13930  4.404 km2

Starting 15 of 45: RGI60-01.09624  4.487 km2

Starting 16 of 45: RGI60-01.15516  4.764 km2

Starting 17 of 45: RGI60-01.21721  6.422 km2

Starting 18 of 45: RGI60-01.10255  7.262 km2

Starting 19 of 45: RGI60-01.12165  7.969 km2

Starting 20 of 45: RGI60-01.05007  9.216 km2

Starting 21 of 45: RGI60-01.01104  9.528 km2

Starting 22 of 45: RGI60-01.12186  11.05 km

In [5]:
obs_df.head()

,Date,observed_initial,terrain_shadows
0,2018-05-02,0.0427,0.0008
1,2018-05-04,0.0000,0.0007
2,2018-05-07,0.8243,0.0006
3,2018-05-09,0.9152,0.0006
4,2018-05-12,0.2016,0.0005
